In [37]:
%pip install langchain
%pip install -qU langchain-community faiss-cpu


%pip install torch torchvision
%pip install -U sentence-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

%pip install -qU langchain_ollama




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -qU langchain_ollama
%pip install -qU pypdf





[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import torch 

# For those with NVIDIA graphics card
torch.cuda.is_available()

# Apple alternative 
torch.backends.mps.is_available()

if torch.cuda.is_available():
    print("CUDA is available")
    hardware = "cuda"

elif torch.backends.mps.is_available():
    print("MPS is available")
    hardware = "mps"

print(f"Using {hardware} for hardware acceleration")


MPS is available
Using mps for hardware acceleration


In [1]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="qwen2.5:1.5b")

response = llm.invoke("What is the capital of Vietnam?")

print(response)

The capital of Vietnam is Hanoi.


In [32]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# files = ["berkeley_opt.pdf", "uscis_opt.pdf"]
loader = PyPDFDirectoryLoader("../examples")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # 1000 characters per chunk
    chunk_overlap=200 # 200 characters overlap between chunks
)

chunks = []

for page in pages:
    chunk_list = text_splitter.split_text(page.page_content)
    for chunk in chunk_list:
        chunks.append(chunk)
print("Number of chunks: ", len(chunks))

print(type(chunks[1]))


Number of chunks:  122
<class 'str'>


## Create Vector Database 
### Cache Setup

In [21]:
import os 
import hashlib 
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import FAISS 


def compute_document_hash(text_chunks: list, embedding_model: str) -> str:
    """
    Compute a hash of the document content using the model name.
    Args: 
        text_chunks: list of text chunks
        embedding_model: name of the embedding model
    Returns:
        hash of the document content
    """

    content = "".join(text_chunks) + embedding_model 
    return hashlib.sha256(content.encode()).hexdigest()[:16]


def create_or_load_vector_store(chunks: list, embeddings: HuggingFaceEmbeddings, cache_dir: str) -> FAISS:
    """ Retrieves a vector store from a list of text chunks using the given embeddings. 
    
    Args: 
        chunks: list of text chunks
        embeddings: embeddings model
        cache_dir: directory to save the vector store
    Returns:
        FAISS object
    """

    os.makedirs(cache_dir, exist_ok=True)


    embedding_model_name = embeddings.model_name
    current_hash = compute_document_hash(chunks, embedding_model_name)

    # Check if cached index exists and contains a valid path 
    hash_file = os.path.join(cache_dir, "content_hash.txt")
    index_file = os.path.join(cache_dir, "index.faiss")

    if os.path.exists(index_file) and os.path.exists(hash_file):
        with open(hash_file, "r") as f:
            cached_hash = f.read().strip()

            if current_hash == cached_hash:
                print("Loading cached FAISS index for document retrieval")
                return FAISS.load_local(
                    folder_path= cache_dir,
                    embeddings=embeddings,
                    allow_dangerous_deserialization=True
                )
            
            else:
                print("Cache invalidated: Documents and embeddings have changed.")
                print("Rebuilding vector store for updated documents...")

                # Delete existing index and hash file 
                for file in os.listdir(cache_dir):
                    os.remove(os.path.join(cache_dir, file))
    else:
        print("No valid cache found. Creating new vector store...")

    
    # Save the new index and hash file 
    print(f"Creating a new vector store with {len(chunks)} document chunks...")
    vector_store = FAISS.from_texts(chunks, embedding = embeddings)

    # Save the new index and hash
    vector_store.save_local(cache_dir)
    with open(hash_file, "w") as f:
        f.write(current_hash)
    
    print("Vector store created and cached successfully!")

    return vector_store


### Retriever Setup

In [38]:
embeddings = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs = {"device": hardware}
)

vector_store = create_or_load_vector_store(chunks, embeddings, "../vector_store")

query = "What is OPT and if when should an international student apply for it?"
results = vector_store.search(query, k = 2, search_type = "similarity")
print(results)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2}
j)

No valid cache found. Creating new vector store...
Creating a new vector store with 122 document chunks...
Vector store created and cached successfully!
[Document(id='990685ef-d0e3-4613-a3e9-4c539fca8297', metadata={}, page_content='Home>Working in the United States>Students and Exchange Visitors>Optional Practical Training (OPT)\nfor F-1 Students\nOptional Practical Training (OPT) for F-1\nStudents\n\uf05aALERT: Please remember that photos submitted to USCIS must be unmounted and unretouched.\nUnretouched means the photos must not be edited or digitally enhanced.\xa0The submission of any\nmounted or retouched images will delay the processing of your application and may prompt USCIS\nto require you to visit an Application Support Center to verify your identity.\nOptional practical training (OPT) is temporary employment that is directly related to an F-1 studentʼs major\narea of study. Eligible students can apply to receive up to 12 months of OPT employment authorization before\ncomplet

In [40]:
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(result.page_content)
    print("\n")




Home>Working in the United States>Students and Exchange Visitors>Optional Practical Training (OPT)
for F-1 Students
Optional Practical Training (OPT) for F-1
Students
ALERT: Please remember that photos submitted to USCIS must be unmounted and unretouched.
Unretouched means the photos must not be edited or digitally enhanced. The submission of any
mounted or retouched images will delay the processing of your application and may prompt USCIS
to require you to visit an Application Support Center to verify your identity.
Optional practical training (OPT) is temporary employment that is directly related to an F-1 studentʼs major
area of study. Eligible students can apply to receive up to 12 months of OPT employment authorization before
completing their academic studies (pre-completion) and/or after completing their academic studies (post-
completion). However, all periods of pre-completion OPT will be deducted from the available period of post-
completion OPT.
Types of OPT


Must apply wit